<a href="https://colab.research.google.com/github/sushirito/Hg2/blob/main/Diffusion_Regions_Dependence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np

k_B = 1.380649e-23  # Boltzmann constant (J/K)
r = (0.102 + 0.168) * 1e-9  # Effective radius (m)

# Lucas method parameters for water
P_C = 22.064e6  # Critical pressure (Pa)
omega = 0.344  # Acentric factor

def dynamic_viscosity(T_K):
    """Compute viscosity at saturated liquid conditions."""
    return 2.414e-5 * 10**(247.8 / (T_K - 140))

def reduced_pressure(P, T_K):
    """Compute reduced pressure using Antoine equation."""
    P_VP = 10**(5.053988 - (1647.6 / (T_K + 226))) * 101325
    return (P - P_VP) / P_C

def lucas_coefficients(T_R):
    """Compute Lucas method coefficients."""
    A = 0.9991 - (4.674e-4 / (1.0523 * T_R - 0.3877 - 1.0513))
    D = 0.3257 / (1.0039 - T_R**2.573)**0.2096 - 0.2086
    C = -0.07921 + 2.1616 * T_R - 13.4040 * T_R**2 + 44.1706 * T_R**3 \
        - 84.8291 * T_R**4 + 96.1209 * T_R**5 - 59.8127 * T_R**6 + 15.6719 * T_R**7
    return A, D, C

def adjusted_viscosity(mu_SL, delta_P_R, A, D, C):
    """Adjust viscosity using the Lucas method."""
    return mu_SL * (1 + D * delta_P_R * A / 2.118) / (1 + C * omega * delta_P_R)

def diffusion_coefficient(T_K, r, viscosity):
    """Compute diffusion coefficient using Einstein-Stokes equation."""
    return k_B * T_K / (6 * np.pi * viscosity * r)

# Data
locations = ["Ross Sea", "Lake Superior", "Kodakainal", "Amazon", "Guadalupe"]
temperatures_C = [-27.90, 1.67, 17.30, 27.78, 21.60]
pressures_Pa = [98780.0, 101325.0, 101660.0, 101325.0, 102608.0]

temperatures_K = [T + 273.15 for T in temperatures_C]

# Compute values and display
print("Location         | Temp (°C) | Pressure (Pa) | Viscosity (Pa·s) | Diffusivity (m²/s)")
print("--------------------------------------------------------------------------------")

for loc, T_C, T_K, P in zip(locations, temperatures_C, temperatures_K, pressures_Pa):
    T_R = T_K / 647.3  # Reduced temperature
    mu_SL = dynamic_viscosity(T_K)
    A, D, C = lucas_coefficients(T_R)
    delta_P_R = reduced_pressure(P, T_K)
    mu_adjusted = adjusted_viscosity(mu_SL, delta_P_R, A, D, C)
    D = diffusion_coefficient(T_K, r, mu_adjusted)
    print(f"{loc:<15} | {T_C:<10.2f} | {P:<12.1f} | {mu_adjusted:<15.6e} | {D:<15.6e}")


Location         | Temp (°C) | Pressure (Pa) | Viscosity (Pa·s) | Diffusivity (m²/s)
--------------------------------------------------------------------------------
Ross Sea        | -27.90     | 98780.0      | 5.426593e-03    | 1.226027e-10   
Lake Superior   | 1.67       | 101325.0     | 1.645817e-03    | 4.529865e-10   
Kodakainal      | 17.30      | 101660.0     | 1.057282e-03    | 7.452445e-10   
Amazon          | 27.78      | 101325.0     | 8.240478e-04    | 9.906756e-10   
Guadalupe       | 21.60      | 102608.0     | 9.506893e-04    | 8.410729e-10   
